<a href="https://colab.research.google.com/github/Indrejue/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Antony_Farag_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [27]:
#all of my imports
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas_profiling
from sklearn.metrics import mean_absolute_error
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [2]:


# Read New York City property sales data
re_nyc = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',parse_dates=['SALE DATE'], index_col='SALE DATE')

# Change column names: replace spaces with underscores
re_nyc.columns = [col.replace(' ', '_') for col in re_nyc]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
re_nyc['SALE_PRICE'] = (
    re_nyc['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
re_nyc.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
2019-01-01,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
2019-01-01,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0
2019-01-01,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0
2019-01-01,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0


In [4]:
re_nyc['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS                       5061
02 TWO FAMILY DWELLINGS                       4567
10 COOPS - ELEVATOR APARTMENTS                3471
13 CONDOS - ELEVATOR APARTMENTS               3339
03 THREE FAMILY DWELLINGS                     1438
07 RENTALS - WALKUP APARTMENTS                 807
09 COOPS - WALKUP APARTMENTS                   672
15 CONDOS - 2-10 UNIT RESIDENTIAL              421
04 TAX CLASS 1 CONDOS                          418
44 CONDO PARKING                               366
17 CONDO COOPS                                 300
22 STORE BUILDINGS                             288
05 TAX CLASS 1 VACANT LAND                     288
12 CONDOS - WALKUP APARTMENTS                  256
14 RENTALS - 4-10 UNIT                         200
29 COMMERCIAL GARAGES                          147
08 RENTALS - ELEVATOR APARTMENTS               120
30 WAREHOUSES                                  105
21 OFFICE BUILDINGS                             96
43 CONDO OFFICE BUILDINGS      

In [5]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
re_nyc['BOROUGH'] = re_nyc['BOROUGH'].astype(str)

In [6]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = re_nyc['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
re_nyc.loc[~re_nyc['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [7]:
#seperating out the single family dwelings from the rest of the data that is between 100k and 2 Million
re_nyc = re_nyc[(re_nyc['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')& 
          (re_nyc['SALE_PRICE'] <=2000000)& 
          (re_nyc['SALE_PRICE'] >=100000)]


In [8]:
#turn land square feet column into an integer value by removin the commas
re_nyc['LAND_SQUARE_FEET'] = (
    re_nyc['LAND_SQUARE_FEET']
    .str.replace(',','')
    .astype(int))

In [9]:
re_nyc.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,1,A5,620000


In [10]:
#identify object columns to be droped
re_nyc.select_dtypes('object').nunique()

BOROUGH                              5
NEIGHBORHOOD                         7
BUILDING_CLASS_CATEGORY              1
TAX_CLASS_AT_PRESENT                 2
BUILDING_CLASS_AT_PRESENT           13
ADDRESS                           3148
APARTMENT_NUMBER                     1
BUILDING_CLASS_AT_TIME_OF_SALE      11
dtype: int64

In [11]:
#identify int and float columns that need to be elminated either because of all nans or not ints
re_nyc.select_dtypes(exclude='object').info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3164 entries, 2019-01-01 to 2019-04-30
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   BLOCK                      3164 non-null   int64  
 1   LOT                        3164 non-null   int64  
 2   EASE-MENT                  0 non-null      float64
 3   ZIP_CODE                   3164 non-null   float64
 4   RESIDENTIAL_UNITS          3164 non-null   float64
 5   COMMERCIAL_UNITS           3164 non-null   float64
 6   TOTAL_UNITS                3164 non-null   float64
 7   LAND_SQUARE_FEET           3164 non-null   int64  
 8   GROSS_SQUARE_FEET          3164 non-null   float64
 9   YEAR_BUILT                 3164 non-null   float64
 10  TAX_CLASS_AT_TIME_OF_SALE  3164 non-null   int64  
 11  SALE_PRICE                 3164 non-null   int64  
dtypes: float64(7), int64(5)
memory usage: 321.3 KB


In [12]:
re_nyc = re_nyc.drop(columns=['EASE-MENT','APARTMENT_NUMBER','ZIP_CODE','ADDRESS','APARTMENT_NUMBER','BUILDING_CLASS_CATEGORY'])
re_nyc.shape

(3164, 15)

In [13]:
re_nyc.head()

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,1,5495,801,A9,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,1,7918,72,A1,1.0,0.0,1.0,4000,2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,1,4210,19,A1,1.0,0.0,1.0,3500,2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,1,5212,69,A1,1.0,0.0,1.0,4000,2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,1,7930,121,A5,1.0,0.0,1.0,1710,1872.0,1940.0,1,A5,620000


In [14]:
#establish my target and parameter data
target = 'SALE_PRICE'
y = re_nyc[target]
X = re_nyc.drop(columns=target)

In [16]:
# split my data by date
cutoff = '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [19]:
# prepare a baseline test array
y_pred = [y_train.mean()] * len(y_train)
y_pred[:5]

[621011.2721493841,
 621011.2721493841,
 621011.2721493841,
 621011.2721493841,
 621011.2721493841]

In [22]:
#take the Mean absolute error of the baseline
mean_absolute_error(y_train, y_pred)

216497.12357684076

In [24]:
#transform catigorical data into something that math can be applied to
ohe = OneHotEncoder(use_cat_names=True)
ohe.fit(X_train)
XT_train = ohe.transform(X_train)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



In [26]:
#apply standard scaler to the data to normalize it.
ss = StandardScaler()
ss.fit(XT_train)
XTT_train = ss.transform(XT_train)

In [37]:
#use ridge regression on the data and test to see the fit
model = Ridge(alpha=5, normalize=False)
model.fit(XTT_train, y_train)

Ridge(alpha=5, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [38]:
mean_absolute_error(y_train, model.predict(XTT_train))

149948.46473277363

In [39]:
#apply onehot encoding and standard scaler to the test data
XT_test = ohe.transform(X_test)
XTT_test = ss.transform(XT_test)


In [40]:
#apply ridge reggresion on test data and see the fit
mean_absolute_error(y_test, model.predict(XTT_test))

157907.24564072242